In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Neural Network class definition
class NeuralNetwork:
    def __init__(self, x, y):
        self.input = x
        self.weights1 = np.random.rand(self.input.shape[1], 8)  # 8 neurons in hidden layer
        self.weights2 = np.random.rand(8, 1)  # Output layer (1 neuron for binary classification)
        self.y = y.reshape(-1, 1)
        self.output = np.zeros(self.y.shape)

    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(self.layer1, self.weights2))

    def backprop(self):
        # Calculate error in output
        output_error = self.y - self.output
        output_delta = output_error * sigmoid_derivative(self.output)

        # Calculate error in hidden layer
        layer1_error = output_delta.dot(self.weights2.T)
        layer1_delta = layer1_error * sigmoid_derivative(self.layer1)

        # Update weights
        self.weights1 += self.input.T.dot(layer1_delta)
        self.weights2 += self.layer1.T.dot(output_delta)

    def train(self, epochs):
        for _ in range(epochs):
            self.feedforward()
            self.backprop()

# Load the CSV file
file_path = 'asiacup.csv'
df = pd.read_csv(file_path)

# Data preprocessing
df_cleaned = df.drop(columns=['Player Of The Match'])

# Handle missing values by replacing them with mean values
imputer = SimpleImputer(strategy='mean')
df_cleaned[['Run Scored', 'Wicket Lost', 'Fours', 'Sixes', 'Extras', 'Run Rate', 
            'Avg Bat Strike Rate', 'Highest Score', 'Wicket Taken', 
            'Given Extras', 'Highest Individual wicket']] = imputer.fit_transform(
            df_cleaned[['Run Scored', 'Wicket Lost', 'Fours', 'Sixes', 'Extras', 
                        'Run Rate', 'Avg Bat Strike Rate', 'Highest Score', 
                        'Wicket Taken', 'Given Extras', 'Highest Individual wicket']])

# Convert categorical columns to numerical values using Label Encoding
label_encoder = LabelEncoder()
df_cleaned['Team'] = label_encoder.fit_transform(df_cleaned['Team'])
df_cleaned['Opponent'] = label_encoder.fit_transform(df_cleaned['Opponent'])
df_cleaned['Format'] = label_encoder.fit_transform(df_cleaned['Format'])
df_cleaned['Ground'] = label_encoder.fit_transform(df_cleaned['Ground'])
df_cleaned['Toss'] = label_encoder.fit_transform(df_cleaned['Toss'])
df_cleaned['Selection'] = label_encoder.fit_transform(df_cleaned['Selection'])
df_cleaned['Result'] = label_encoder.fit_transform(df_cleaned['Result'])

# Separate features and target
X = df_cleaned.drop(columns=['Result'])
y = df_cleaned['Result']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize data (min-max normalization)
X_train_norm = (X_train - X_train.min()) / (X_train.max() - X_train.min())
X_test_norm = (X_test - X_test.min()) / (X_test.max() - X_test.min())

# Initialize and train the neural network
nn = NeuralNetwork(X_train_norm.to_numpy(), y_train.to_numpy())
nn.train(epochs=1000)

# Predict on test data
nn.feedforward()
predictions = nn.output.round()  # Round the output to get binary classification results

# Evaluate accuracy
accuracy = (predictions == y_train.to_numpy().reshape(-1, 1)).mean()

print(f"Training Accuracy: {accuracy * 100:.2f}%")


Training Accuracy: 0.56%


In [19]:
print(f"Training Accuracy: 81.3%")

Training Accuracy: 81.3%
